In [1]:
import os
import torch
import random
import warnings
import numpy as np
import torchvision
import torch.nn as nn
from tqdm import tqdm
from copy import deepcopy
from kornia import augmentation
import torch.nn.functional as F
import matplotlib.pyplot as plt
from diffusers import UNet2DModel
from datasets import load_dataset
from torchvision import transforms
from diffusers import DDPMScheduler
from PIL import ImageDraw, ImageFont, Image
from datasets import load_dataset, load_metric
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings("ignore")

/work/pi_dhruveshpate_umass_edu/aamballa_umass_edu/.conda/envs/DFME/lib/python3.9/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


### Define Generator, Victim and Stolen

In [3]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, features=[64, 128, 256, 512]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Downsampling
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Upsampling
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2)
            )
            self.ups.append(DoubleConv(feature * 2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        # Downsampling
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        # Upsampling
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx // 2]

            if x.shape != skip_connection.shape:
                x = nn.functional.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx + 1](concat_skip)

        return self.final_conv(x)

In [4]:
def get_generator(args):
    class MultiStepGenerator(nn.Module):
        def __init__(self, args):
            super(MultiStepGenerator, self).__init__()
            self.unet =UNet(in_channels=args.img_c, out_channels=args.img_c)
            self.num_timesteps = args.num_timesteps
        
        def forward(self, noise):
            images = []
            current_image = noise
    
            for _ in range(self.num_timesteps):
                current_image = self.unet(current_image)
                images.append(current_image)
            
            return images

    return MultiStepGenerator(args).to(device)

In [5]:
# class Net(nn.Module):
#         def __init__(self):
#             super(Net, self).__init__()
#             self.conv1 = nn.Conv2d(1, 32, 3, 1)
#             self.conv2 = nn.Conv2d(32, 64, 3, 1)
#             self.fc1 = nn.Linear(12544, 128)
#             self.fc2 = nn.Linear(128, 10)
    
#         def forward(self, x):
#             x = F.relu(self.conv1(x))
#             x = F.relu(self.conv2(x))
#             x = F.max_pool2d(x, 2)
#             x = torch.flatten(x, 1)
#             x = F.relu(self.fc1(x))
#             x = self.fc2(x)
#             return x

In [6]:
class NetS(nn.Module):
        def __init__(self):
            super(NetS, self).__init__()
            self.conv1 = nn.Conv2d(1, 32, 3, 1)
            self.conv2 = nn.Conv2d(32, 64, 3, 1)
            self.fc1 = nn.Linear(12544, 128)
            self.fc2 = nn.Linear(128, 10)
            self.bn1 = nn.BatchNorm2d(32)
            self.bn2 = nn.BatchNorm2d(64)
    
        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = self.bn1(x)
            x = F.relu(self.conv2(x))
            x = self.bn2(x)
            x = F.max_pool2d(x, 2)
            x = torch.flatten(x, 1)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return x

In [7]:
def get_victim_clone(args):
    
    student = NetS().to(args.device)
    victim = torch.load(args.victim_path, map_location=args.device)
    return victim , student

### Utilites

In [8]:
class Args:
    def __init__(self, device, seed, epochs, batch_size,num_timesteps, img_n, img_c, momentum,
                 img_w, img_h, lr_G, lr_C, lr_hee, lr_z, weight_decay, N_G, N_C, 
                 steps_hee, grad_accumulation_steps, std_aug, lam, label_smooth_factor,
                 victim_path, N_classes, debug, save_dir):
        
        self.device = device
        self.seed = seed
        self.epochs = epochs
        self.batch_size = batch_size
        self.num_timesteps = num_timesteps
        self.img_n = img_n
        self.img_c = img_c
        self.img_w = img_w
        self.img_h = img_h
        self.lr_G = lr_G
        self.lr_C = lr_C
        self.lr_hee = lr_hee
        self.lr_z = lr_z
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.N_G = N_G
        self.N_C = N_C
        self.steps_hee = steps_hee
        self.grad_accumulation_steps = grad_accumulation_steps
        self.std_aug = std_aug
        self.lam = lam
        self.victim_path = victim_path
        self.N_classes = N_classes
        self.debug = debug
        self.victim_path = victim_path
        self.label_smooth_factor = label_smooth_factor
        self.save_dir = save_dir

In [9]:
class FakeDataset(torch.utils.data.Dataset):
    """Some Information about FakeDataset"""

    def __init__(self, root="", transform=None):
        super(FakeDataset, self).__init__()

        self.transform = transform

        history_images = np.load(os.path.join(root, "fake_images.npy"))
        self.images = torch.from_numpy(history_images)

    def __getitem__(self, index):
        image = self.images[index]

        return image

    def __len__(self):
        return len(self.images)

In [10]:
class TensorDataset(Dataset):
    def __init__(self, tensor):
        self.tensor = tensor

    def __len__(self):
        return len(self.tensor)

    def __getitem__(self, idx):
        return self.tensor[idx]

In [11]:
class DataIter(object):
    def __init__(self, dataloader):
        self.dataloader = dataloader
        self._iter = iter(self.dataloader)

    def next(self):
        try:
            data = next(self._iter)
        except StopIteration:
            self._iter = iter(self.dataloader)
            data = next(self._iter)
        return data

In [12]:
def get_standard_augment(img_w, img_h):
    std_aug = augmentation.container.ImageSequential(
    augmentation.RandomCrop(size=[img_w, img_h], padding=4),
    augmentation.RandomHorizontalFlip(),
)
    return std_aug

In [13]:
def strong_aug(image):
    device = image.device
    image = TF.center_crop(
        image,
        [int(32.0 * random.uniform(0.95, 1.0)), int(32.0 * random.uniform(0.95, 1.0))],
    )
    image = TF.resize(image, [32, 32])
    noise = torch.randn_like(image).to(device) * 0.001
    image = torch.clamp(image + noise, 0.0, 1.0)
    if random.uniform(0, 1) > 0.5:
        image = TF.vflip(image)
    if random.uniform(0, 1) > 0.5:
        image = TF.hflip(image)
    angles = [-15, 0, 15]
    angle = random.choice(angles)
    image = TF.rotate(image, angle)
    return image

In [15]:
def smooth_one_hot(args, true_labels):
    """
    if smoothing == 0, it's one-hot method
    if 0 < smoothing < 1, it's smooth method
    """
    device = true_labels.device
    true_labels = torch.nn.functional.one_hot(true_labels, args.N_classes).detach().cpu()
    assert 0 <= args.label_smooth_factor < 1
    confidence = 1.0 - args.label_smooth_factor
    label_shape = torch.Size((true_labels.size(0), args.N_classes))
    with torch.no_grad():
        true_dist = torch.empty(size=label_shape, device=true_labels.device)
        true_dist.fill_(args.label_smooth_factor / (args.N_classes - 1))
        _, index = torch.max(true_labels, 1)

        true_dist.scatter_(1, torch.LongTensor(index.unsqueeze(1)), confidence)
    return true_dist.to(device)

In [16]:
def evaluate(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images.to(device))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.to(device)).sum().item()
    
    print(f'Accuracy of the network on the 10,000 test images: {100 * correct / total:.2f}%')


In [17]:
def save_batch_fake(args, images, epoch):
    images = images.detach().cpu().numpy()
    images_filename = os.path.join(args.save_dir, "fake_images.npy")

    if epoch > 1:
        org_images = np.load(images_filename)

        images = np.concatenate((org_images, images), 0)

    np.save(images_filename, images)

In [18]:
# Function to calculate the Gram matrix
def gram_matrix(x):
    return torch.matmul(x, x.t())

### DFME++ Attack

In [19]:
# Loss function: L1 loss between student and teacher Gram matrices
def l1_loss_between_grams(student_probs, teacher_probs):  # Student prob : time steps, classes. 
    student_gram = gram_matrix(student_probs)
    teacher_gram = gram_matrix(teacher_probs)
    return F.l1_loss(student_gram, teacher_gram)

In [25]:
class Entropy_Loss(nn.Module):
    def __init__(self, reduction="mean"):
        super(Entropy_Loss, self).__init__()
        self.reduction = reduction

    def forward(self, x):  # x should be logits
        b = F.softmax(x, dim=1) * F.log_softmax(x, dim=1)
        b = 1.0 * b.sum(dim=1)
        if self.reduction == "mean":
            return b.mean()
        elif self.reduction == "sum":
            return b.sum()
        elif self.reduction == "none":
            return b

In [22]:
# def cross_entropy(outputs, smooth_labels):
#     loss = torch.nn.KLDivLoss(reduction="batchmean")
#     return loss(F.log_softmax(outputs, dim=1), smooth_labels)

In [23]:
# def generate_hee(args, model, x):
#     model.eval()
#     x_hee = x.detach() + 0.001 * torch.torch.randn(x.shape).to(args.device).detach()
#     for _ in range(args.steps_hee):
#         x_hee.requires_grad_()
#         with torch.enable_grad():
#             pred = model(x_hee)
#             loss = Entropy_Loss(reduction="mean")(pred)
#         grad = torch.autograd.grad(loss, [x_hee])[0] 
#         x_hee = x_hee.detach() + args.lr_hee * torch.sign(grad.detach())
#         x_hee = torch.clamp(x_hee, 0.0, 1.0)
#     model.train()

#     return x_hee

In [44]:
def train_generator(args, generator_model, clone_model, epoch):

    generator_model.train()
    clone_model.eval()
    
    best_fake = None
    best_loss = 1e6
    
    if args.debug: print('Debug(train_generator) :-> Generating Images using Generator Model')
    z = torch.randn(size=(args.img_n, args.img_c,args.img_w, args.img_h )).to(args.device)
    z.requires_grad = True
    
    optimizer_G = torch.optim.Adam([{"params": generator_model.parameters()}, {"params": [z], "lr": args.lr_z}], lr=args.lr_G, betas=[0.5, 0.999])
    # pseudo_y = torch.randint(low=0, high=args.N_classes, size=(args.img_n,)).to(args.device)
    # soft_labels = smooth_one_hot(args, pseudo_y)
    criterion = Entropy_Loss()
    
    if args.debug: print('Debug(train_generator) :-> Starting Generator Training')
    for step in range(args.N_G):
    
        fake = torch.stack(generator_model(z)).squeeze(1)
        aug_fake = args.std_aug(fake)  # data augmentation to fake images. 
        
        logits = clone_model(aug_fake)
        # print(logits.shape)
        loss = criterion(logits)
    
    
        with torch.no_grad():
            if best_loss > loss.item() or best_fake is None:
                best_loss = loss.item()
                best_fake = fake
        
        optimizer_G.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm(clone_model.parameters(), max_norm = 1)
        optimizer_G.step()
    print(f"Generator Step {step} average loss: {loss}")
    save_batch_fake(args, best_fake.data, epoch)
    if args.debug: print('Debug(train_generator) :-> Generator Training Ended')
    return generator_model

In [58]:
def train_clone(args, generator_model, clone_model, victim_model, optimizer_C):

    generator_model.eval()
    clone_model.train()
    victim_model.eval()
    
    dataset = FakeDataset(root=args.save_dir)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=True,
    )
    data_iter = DataIter(data_loader)
    
    victim_classes_fired = np.zeros(args.N_classes)
    clone_classes_fired = np.zeros(args.N_classes)
    losses = []
    if args.debug: print('Debug(train_clone) :-> Starting Clone Model Training')        
    for step in range(args.N_C):
        fake = data_iter.next().to(args.device)
        aug_fake = args.std_aug(fake)  # already applied before saving
        # fake_hee = generate_hee(args, clone_model,strong_aug(aug_fake))
        
        logits_T = victim_model(aug_fake).detach() 
        # print(logits.T)
        hard_labels = logits_T.topk(1, 1)[1].reshape(-1)
        victim_probs = F.softmax(logits_T)
        np.add.at(victim_classes_fired, hard_labels.cpu().numpy() , 1)
        
        logits_C = clone_model(aug_fake)
        clone_probs = F.softmax(logits_C)
        np.add.at(clone_classes_fired, logits_C.topk(1, 1)[1].reshape(-1).cpu().numpy() , 1)
        
        loss = l1_loss_between_grams(clone_probs, victim_probs)
        losses.append(loss.item())
        optimizer_C.zero_grad()
        loss.backward()
        optimizer_C.step()
        if step % 200 == 0:
            print(f"Clone Steps {step} average loss: {sum(losses[-len(data_loader):])/len(data_loader)}")
            print(f"Clone Steps {step} Victim Classes Fired: {victim_classes_fired})")
            print(f"Clone Steps {step} Clone Classes Fired: {clone_classes_fired})")
    if args.debug: print('Debug(train_clone) :-> Clone Model Training Ended') 
    return (generator_model, clone_model, victim_model, optimizer_C)

In [59]:
debug = 1 #To debug code
device = torch.device('cuda') #device placement cpu or gpu
seed = 10 #seed for consistent result
epochs = 10 #number of epochs to train
batch_size = 256 #batchsize
num_timesteps = 100 #timestamps in multistepgenerator
img_n = 1 
img_c = 1 #image channel
img_w = 32 #image size
img_h = 32 #image size
lr_z = 0.01 #learning rate of latent code
lr_G = 0.001 #learning rate of Generator
lr_C = 0.001 #learing rate of clone model
lr_hee = 0.03 #perturb number of steps
weight_decay = 1e-4 #Optimizer parameter: decay's weight update
momentum = 0.9 #Optimizer parameter: Remeber past information 1/momentum times
N_G = 100 #Diffuser train steps
N_C = 500 #Clone model steps 
steps_hee = 10 #number of epochs to train
grad_accumulation_steps = 16 #update model after no.of steps
std_aug = get_standard_augment(img_w, img_h) #standard augmentation: flip, crop
lam = 0.009 #hyperparameter for balancing two loss terms in diffuser
victim_path = "Model/fashion_mnist.pt" #clone model path
N_classes = 10 #No.of classes to predict
label_smooth_factor = 0.2 #Hard to soft logits
save_dir = r'Gen_Imgs/Gram' #Folder to save genrator images

In [60]:
args = Args(
        debug = debug,
        device = device,
        seed = seed,
        epochs = epochs,
        batch_size = batch_size,
        num_timesteps = num_timesteps,
        img_n = img_n,
        img_c = img_c,
        img_w = img_w,
        img_h = img_h,
        lr_G = lr_G,
        lr_C = lr_C,
        lr_z = lr_z,
        lr_hee = lr_hee,
        weight_decay = weight_decay,
        momentum = momentum,
        N_G = N_G,
        N_C = N_C,
        steps_hee = steps_hee,
        grad_accumulation_steps = grad_accumulation_steps,
        std_aug = std_aug,
        lam = lam,
        victim_path = victim_path,
        N_classes = N_classes,
        label_smooth_factor = label_smooth_factor,
        save_dir = save_dir
    )


In [29]:
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [63]:
generator_model =  get_generator(args)
victim_model, clone_model = get_victim_clone(args)

In [31]:
optimizer_C = torch.optim.AdamW(
        clone_model.parameters(),
        lr=args.lr_C,
        weight_decay=args.weight_decay,
    )
scheduler_lr = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer_C, args.epochs, eta_min=2e-4
)


In [32]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((32, 32))])
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

In [33]:
testset

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
           )

In [64]:
evaluate(victim_model, testloader), evaluate(clone_model, testloader)

Accuracy of the network on the 10,000 test images: 92.07%
Accuracy of the network on the 10,000 test images: 13.11%


(None, None)

In [65]:
 for epoch in tqdm(range(1, args.epochs + 1)):
     
    generator_model = train_generator(args, generator_model, clone_model, epoch)
    print('-'*50)
    generator_model, clone_model, victim_model, optimizer_C = train_clone(args, generator_model, clone_model, victim_model, optimizer_C)
    print('-'*50)
    print(scheduler_lr.get_last_lr())
    print()
    evaluate(clone_model, testloader)
    print()
    scheduler_lr.step()


  0%|                                                                                                                                                                                    | 0/10 [00:00<?, ?it/s]

Debug(train_generator) :-> Generating Images using Generator Model
Debug(train_generator) :-> Starting Generator Training
Generator Step 99 average loss: -0.3733247220516205
Debug(train_generator) :-> Generator Training Ended
--------------------------------------------------
Debug(train_clone) :-> Starting Clone Model Training
Clone Steps 0 average loss: 0.8021481037139893
Clone Steps 0 Victim Classes Fired: [ 0.  4.  0.  0.  0. 94.  1.  0.  1.  0.])
Clone Steps 0 Clone Classes Fired: [ 1. 13.  3.  1. 21. 22. 37.  0.  0.  2.])
Clone Steps 200 average loss: 0.7305222749710083
Clone Steps 200 Victim Classes Fired: [9.0000e+00 8.2200e+02 1.9000e+01 0.0000e+00 0.0000e+00 1.8591e+04
 3.8000e+01 6.0000e+00 5.4400e+02 7.1000e+01])
Clone Steps 200 Clone Classes Fired: [3.450e+02 4.068e+03 4.270e+02 1.640e+02 3.366e+03 3.856e+03 6.948e+03
 2.860e+02 5.000e+00 6.350e+02])
Clone Steps 400 average loss: 0.7488370537757874
Clone Steps 400 Victim Classes Fired: [1.5000e+01 1.6810e+03 3.5000e+01 0.0

 10%|█████████████████▏                                                                                                                                                          | 1/10 [01:08<10:15, 68.42s/it]

Accuracy of the network on the 10,000 test images: 13.11%

Debug(train_generator) :-> Generating Images using Generator Model
Debug(train_generator) :-> Starting Generator Training
Generator Step 99 average loss: -0.3028698265552521
Debug(train_generator) :-> Generator Training Ended
--------------------------------------------------
Debug(train_clone) :-> Starting Clone Model Training
Clone Steps 0 average loss: 0.7174571752548218
Clone Steps 0 Victim Classes Fired: [  0.   8.   0.   0.   0. 176.   0.   0.  14.   2.])
Clone Steps 0 Clone Classes Fired: [ 4. 42.  4.  1. 39. 44. 59.  3.  0.  4.])
Clone Steps 200 average loss: 0.8196738958358765
Clone Steps 200 Victim Classes Fired: [5.700e+01 1.380e+03 4.700e+01 1.250e+02 5.000e+00 3.677e+04 7.800e+01
 1.100e+01 1.521e+03 2.060e+02])
Clone Steps 200 Clone Classes Fired: [4.3800e+02 8.5320e+03 7.1600e+02 2.1900e+02 7.5510e+03 7.9300e+03
 1.2931e+04 7.4000e+02 6.0000e+00 1.1370e+03])
Clone Steps 400 average loss: 0.6983717679977417
Clone 

 20%|██████████████████████████████████▍                                                                                                                                         | 2/10 [02:23<09:36, 72.11s/it]

...

Accuracy of the network on the 10,000 test images: 13.11%

Debug(train_generator) :-> Generating Images using Generator Model
Debug(train_generator) :-> Starting Generator Training
Generator Step 99 average loss: -0.2772515118122101
Debug(train_generator) :-> Generator Training Ended
--------------------------------------------------
Debug(train_clone) :-> Starting Clone Model Training
Clone Steps 0 average loss: 0.17009992897510529
Clone Steps 0 Victim Classes Fired: [  0.   2.   0.   0.   0. 218.   3.   1.  32.   0.])
Clone Steps 0 Clone Classes Fired: [ 3. 42.  6.  2. 43. 43. 95. 13.  1.  8.])
Clone Steps 200 average loss: 0.6697021722793579
Clone Steps 200 Victim Classes Fired: [2.4100e+02 6.4600e+02 4.1000e+01 2.2700e+02 1.0000e+00 3.7192e+04
 3.8600e+02 5.9000e+01 6.1070e+03 3.5600e+02])
Clone Steps 200 Clone Classes Fired: [ 1070.  7153.  1690.   586.  7058.  8183. 15655.  1940.   105.  1816.])
Clone Steps 400 average loss: 0.63919498026371
Clone Steps 400 Victim Classes Fired: 

 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                 | 9/10 [10:53<01:12, 72.70s/it]

Accuracy of the network on the 10,000 test images: 13.11%

Debug(train_generator) :-> Generating Images using Generator Model
Debug(train_generator) :-> Starting Generator Training
Generator Step 99 average loss: -0.28388601541519165
Debug(train_generator) :-> Generator Training Ended
--------------------------------------------------
Debug(train_clone) :-> Starting Clone Model Training
Clone Steps 0 average loss: 0.16461628675460815
Clone Steps 0 Victim Classes Fired: [  1.   4.   0.   0.   0. 215.   3.   3.  27.   3.])
Clone Steps 0 Clone Classes Fired: [  9.  34.   7.   1.  40.  41. 103.  12.   0.   9.])
Clone Steps 200 average loss: 0.6442815512418747
Clone Steps 200 Victim Classes Fired: [  257.   750.    44.   226.     0. 41448.   399.    66.  6655.   411.])
Clone Steps 200 Clone Classes Fired: [ 1211.  8019.  1912.   582.  7746.  8969. 17600.  2116.   103.  1998.])
Clone Steps 400 average loss: 0.6427726745605469
Clone Steps 400 Victim Classes Fired: [5.4000e+02 1.4860e+03 8.000

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [12:08<00:00, 72.82s/it]

Accuracy of the network on the 10,000 test images: 13.11%

